In [ ]:
import os
project_dir = "/Volumes/mnt/Workspace/ai-trading-agents"
os.chdir(project_dir)

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import dotenv
dotenv.load_dotenv()

In [ ]:
from src.context import build_context
from src.agents import build_fundamental_analyst_agent

In [ ]:
run_id = "b13e74d2-baef-40e0-a5f4-006f7019a13b"
model_name = "deepseek"

In [ ]:
context, agent = await build_fundamental_analyst_agent(model_name, run_id)

In [ ]:
cheif_investment_officer_prompt = """
Now, your investment team is looking for a comprehensive fundamental analysis report on the ticker CRCL.
You're dedicated to providing accurate and up-to-date financial data, valuation metrics, growth prospects, and risk factors to help them make informed investment decisions.
Finally, return the final report consolidated from the above information.
"""

In [ ]:
response = await agent.ainvoke(
    {"messages": [{"role": "user", "content": cheif_investment_officer_prompt}]},
    context=context,
    stream_mode="values",
)

In [ ]:
response['messages']

In [ ]:
from src import db

In [ ]:
message_json_list = []
for message in response['messages']:
    message_json_list.append(message.model_dump_json())

In [ ]:
await db.connect()
agent_message = await db.prisma.agentmessage.delete_many(
    where={
        "runId": context.run.id,
    }
)
await db.disconnect()

In [ ]:
message_json_stream_list = []
message_json_list = []
for message in response['messages']:
    message_json_list.append(message.model_dump_json())
    message_json_stream_list.append(message_json_list.copy())

In [ ]:
import asyncio
import random
import json

In [ ]:
await db.connect()

agent_message = await db.prisma.agentmessage.create(
    data={
        "role": "CHIEF_INVESTMENT_OFFICER",
        "botId": context.bot.id,
        "messages": json.dumps(message_json_stream_list[0]),
        "runId": context.run.id,
    }
)

for message_json in message_json_stream_list[1:]:
    await asyncio.sleep(2+random.randint(0, 3))
    agent_message = await db.prisma.agentmessage.update(
        where={
            "id": agent_message.id,
        },
        data={
            "role": "FUNDAMENTAL_ANALYST",
            "botId": context.bot.id,
            "messages": json.dumps(message_json),
            "runId": context.run.id,
        }
    )
await db.disconnect()